<a id="top"></a>
## NIRISS AMI simulation of binary point source AB-Dor and calibrator HD37093


***

## Imports
Describe the libraries we're using here. If there's something unusual, explain what the library is, and why we need it.
- *numpy* to handle array functions
- *astropy.io fits* for accessing FITS files
- *astropy.table Table* for creating tidy tables of the data
- *matplotlib.pyplot* for plotting data

In [1]:
%matplotlib inline

import glob
import io
import os
import sys
import time

from astropy.io import fits
import matplotlib.pyplot as plt
import yaml
import zipfile
import urllib.request

from mirage import imaging_simulator
from mirage.yaml import yaml_generator

# Introduction
*Developer Note:*
Activate your mirage environment and type 'jupyter notebook at the prompt

This notebook creates MIRAGE simulations for binary point source AB-Dor and calibrator HD37093 using the Aperture Masking Interferometry (AMI) mode on JWST NIRISS. The sources are observed with the Non-redundant mask (NRM) in the pupil wheel and filter F480M in the filter wheel at POS1 dither position near the center of SUB80 array.

The current analysis tools are work in progress and do not give correct results if the data has bad pixels. The simulations are therefore created using dark ramps without bad pixels.

### Description of reference files used for simulations

We are using the following non-default reference files to generate the simulation. 
- dark000001_uncal.fits has ngroups = 5 and is used for AB Dor observation that also has ngroups = 5. 
- dark000005_uncal.fits has ngroups = 12 and is used for HD37093 observation that also has ngroups = 12. 
- jwst_niriss_gain_general.fits is the gain file with one value for all pixels 
- jwst_niriss_flat_general.fits is the flat field reference file with no structure 
- jwst_niriss_superbias_sim.fits bias reference file to match the simulation

We are temporarily using local niriss_gridded_psf_library that was created using a new version of non redundant mask. When Mirage gridded psf library gets updated with WebbPSF PSFs that use new version of NRM we can switch back to the default library.



***

### Loading input files and reference files

In [2]:
#boxlink = 'https://data.science.stsci.edu/redirect/JWST/jwst-data_analysis_tools/niriss_ami_binary/ami_mirsim.zip'
#boxfile = './ami_mirsim.zip'

# Download zip file
#if not os.path.exists(boxfile):
#    urllib.request.urlretrieve(boxlink, boxfile)    
#    zf = zipfile.ZipFile(boxfile, 'r')
#    zf.extractall()

### Create output directory to store simulated data

In [3]:
odir = './mirage_sim_data'
if not os.path.exists(odir):
        os.mkdir(odir)
simdata_output_directory = odir

### Define input files needed for simulation

In [4]:
# Define input files
#You can get the xml and pointing files by using File ---> Export option is APT. 
#APT file used niriss_ami_binary_2022.25coords.aptx 
xml_name = './mirage_input_files/niriss_ami_binary_2022.25coords.xml'
pointing_name = './mirage_input_files/niriss_ami_binary_2022.25coords.pointing'


# AB Dor is in field 19 and HD37093 is in field 20 in Kevin's targets.xlsx file
catalogues = {'AB-DOR': {'point_source':  './mirage_input_files/stars_field19_20_combined_allfilters_new.list'
                        },
              'HD-37093': {'point_source': './mirage_input_files/stars_field19_20_combined_allfilters_new.list'
                        }
             }   

### Get the value of PAV3 for each observation. Another way to get PAV3 is from the APT file Reports.

In [5]:
obs1 = 1
pav3_obs1 = yaml_generator.default_obs_v3pa_on_date(pointing_name, obs1, date='2022-04-01')
obs2 = 2
pav3_obs2 = yaml_generator.default_obs_v3pa_on_date(pointing_name, obs2, date='2022-04-01')

roll_angle = {'001':pav3_obs1 , '002': pav3_obs2}

dates = '2022-04-01'
reffile_defaults = 'crds'
datatype = 'raw'

### Run the yaml generator

In [6]:
yam = yaml_generator.SimInput(input_xml = xml_name, pointing_file = pointing_name,
                              catalogs = catalogues, roll_angle = roll_angle,
                              dates = dates, reffile_defaults = reffile_defaults,
                              verbose = True, output_dir = odir,
                              simdata_output_dir = simdata_output_directory,
                              datatype = datatype)

yam.create_inputs()
print("Created yaml files")
time.sleep(2)

#Create yaml files for all observations.
yaml_files = sorted(glob.glob(os.path.join(odir, 'jw*.yaml')))

print(yaml_files)

CRDS_PATH environment variable not set. Setting to /Users/thatte/crds_cache
target_info:
{'AB-DOR': ('05:28:44.9772', '-65:26:51.63'), 'HD-37093': ('05:31:8.2802', '-65:07:41.99')}
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Observation `001` labelled `None` uses template `NirissAmi`
APTObservationParams Dictionary holds 0 entries before reading template
Number of dithers: 1 primary * 1 subpixel = 1
Dictionary read from template has 1 entries.
Found 0.0 tile(s) for observation 001 Observation 1
Found 1 visits with numbers: [1]
APTObservationParams Dictionary holds 1 entries after reading template (+1 entries)
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Observation `002` labelled `None` uses template `NirissAmi`
APTObservationParams Dictionary holds 1 entries before reading template
Number of dithers: 1 primary * 1 subpixel = 1
Dictionary read from template has 1 entries.
Fo

### Update the contents of yaml files and generate data

In [7]:
for file in yaml_files:
    
    # set astrometric reference file to None to use pysiaf
    # To create data without bad pixels use the following reference files.
    with open(file, 'r') as infile:
        yaml_content = yaml.safe_load(infile)
    yaml_content['Reffiles']['astrometric'] = 'None'
    yaml_content['simSignals']['psf_wing_threshold_file'] = 'config'
    yaml_content['Reffiles']['linearized_darkfile'] = 'None'
    yaml_content['simSignals']['psfpath'] = './ref_files_non_default/niriss_gridded_psf_library_newmask'
    yaml_content['Reffiles']['gain'] = './ref_files_non_default/jwst_niriss_gain_general.fits'
    yaml_content['Reffiles']['pixelflat'] = './ref_files_non_default/jwst_niriss_flat_general.fits'
    yaml_content['Reffiles']['superbias'] = './ref_files_non_default/jwst_niriss_superbias_sim.fits'

    
    if "jw01093001" in file:
       yaml_content['Reffiles']['dark'] = './ref_files_non_default/simdarks/dark000001/dark000001_uncal.fits'
    elif "jw01093002" in file:
       yaml_content['Reffiles']['dark'] = './ref_files_non_default/simdarks/dark000005/dark000005_uncal.fits'
    

    modified_file = file.replace('.yaml', '_mod.yaml')
    with io.open(modified_file, 'w') as outfile:
        yaml.dump(yaml_content, outfile, default_flow_style=False)


    print("Updated yaml files. The jw*_mod.yaml files will be used to create data")
    time.sleep(2)
    
    #create data
    t1 = imaging_simulator.ImgSim()
    t1.paramfile = str(modified_file)
    t1.create()

Updated yaml files. The jw*_mod.yaml files will be used to create data


Running catalog_seed_image..

From CRDS, found /Users/thatte/crds_cache/references/jwst/niriss/jwst_niriss_mask_0014.fits as the badpixmask reference file.
From CRDS, found /Users/thatte/crds_cache/references/jwst/niriss/jwst_niriss_linearity_0011.fits as the linearity reference file.
From CRDS, found /Users/thatte/crds_cache/references/jwst/niriss/jwst_niriss_saturation_0010.fits as the saturation reference file.
From CRDS, found /Users/thatte/crds_cache/references/jwst/niriss/jwst_niriss_ipc_0008.fits as the ipc reference file.
Found an existing inverted kernel for this IPC file: /Users/thatte/crds_cache/references/jwst/niriss/Kernel_to_add_IPC_effects_from_jwst_niriss_ipc_0008.fits
From CRDS, found /Users/thatte/crds_cache/references/jwst/niriss/jwst_niriss_area_0011.fits as the pixelAreaMap reference file.
From CRDS, found /Users/thatte/crds_cache/references/jwst/niriss/jwst_niriss_photom_0038.fits as the phot

2020-09-16 10:24:46,419 - CRDS - ERROR -  Error determining best reference for 'pars-dqinitstep'  =   Unknown reference type 'pars-dqinitstep'
2020-09-16 10:24:46,422 - stpipe.dq_init - INFO - DQInitStep instance created.
2020-09-16 10:24:46,518 - stpipe.dq_init - INFO - Step dq_init running with args (<RampModel(1, 5, 2048, 2048) from dark000001_uncal.fits>,).
2020-09-16 10:24:46,520 - stpipe.dq_init - INFO - Step dq_init parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': ''}
2020-09-16 10:24:46,534 - stpipe.dq_init - INFO - Using MASK reference file /Users/thatte/crds_cache/references/jwst/niriss/jwst_niriss_mask_0014.fits


Creating a linearized version of the dark current input ramp
using JWST calibration pipeline.


2020-09-16 10:24:47,042 - stpipe.dq_init - INFO - Step dq_init done
2020-09-16 10:24:47,118 - CRDS - ERROR -  Error determining best reference for 'pars-saturationstep'  =   Unknown reference type 'pars-saturationstep'
2020-09-16 10:24:47,124 - stpipe.saturation - INFO - SaturationStep instance created.
2020-09-16 10:24:47,191 - stpipe.saturation - INFO - Step saturation running with args (<RampModel(1, 5, 2048, 2048) from dark000001_uncal.fits>,).
2020-09-16 10:24:47,194 - stpipe.saturation - INFO - Step saturation parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': ''}
2020-09-16 10:24:47,211 - stpipe.saturation - INFO - Using SATURATION reference file /Users/thatte/crds_cache/references/jwst/niriss/jwst_niriss_saturation_0010.fits
2020-09-16 10:24:47,621 - stpipe.saturation 

Linearized dark shape: (1, 5, 2048, 2048)
Output is NISRAPID, grabbing zero frame from linearized dark
File number 0 will be used for integrations [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64]
Linearized dark frame plus superbias and referencepixel signals, as well as zeroframe, saved to /Users/thatte/niriss/dat_pyinthesky/jdat_notebooks/niriss_ami_binary/mirage_sim_data/jw01093001001_01101_00001_nis_uncal_linear_dark_prep_object.fits. This can be used as input to the observation generator.


Segment 1 out of 1 complete.

Running observation generator....

From CRDS, found /Users/thatte/crds_cache/references/jwst/niriss/jwst_niriss_mask_0014.fits as the badpixmask reference file.
From CRDS, found /Users/thatte/crds_cache/references/jwst/niriss/jwst_niriss_linearity_0011.fits as the linearity reference file.
From CRDS, found /Users/that

...

Linearized dark shape: (1, 12, 2048, 2048)
Output is NISRAPID, grabbing zero frame from linearized dark
File number 0 will be used for integrations [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59]
Linearized dark frame plus superbias and referencepixel signals, as well as zeroframe, saved to /Users/thatte/niriss/dat_pyinthesky/jdat_notebooks/niriss_ami_binary/mirage_sim_data/jw01093002001_01101_00001_nis_uncal_linear_dark_prep_object.fits. This can be used as input to the observation generator.


Segment 1 out of 1 complete.

Running observation generator....

From CRDS, found /Users/thatte/crds_cache/references/jwst/niriss/jwst_niriss_mask_0014.fits as the badpixmask reference file.
From CRDS, found /Users/thatte/crds_cache/references/jwst/niriss/jwst_niriss_linearity_0011.fits as the linearity reference file.
From CRDS, found /Users/thatte/crds_cache/

    Averaging frame 10 into group 10
    Averaging frame 11 into group 11
Integration 24:
    Averaging frame 0 into group 0
    Averaging frame 1 into group 1
    Averaging frame 2 into group 2
    Averaging frame 3 into group 3
    Averaging frame 4 into group 4
    Averaging frame 5 into group 5
    Averaging frame 6 into group 6
    Averaging frame 7 into group 7
    Averaging frame 8 into group 8
    Averaging frame 9 into group 9
    Averaging frame 10 into group 10
    Averaging frame 11 into group 11
Integration 25:
    Averaging frame 0 into group 0
    Averaging frame 1 into group 1
    Averaging frame 2 into group 2
    Averaging frame 3 into group 3
    Averaging frame 4 into group 4
    Averaging frame 5 into group 5
    Averaging frame 6 into group 6
    Averaging frame 7 into group 7
    Averaging frame 8 into group 8
    Averaging frame 9 into group 9
    Averaging frame 10 into group 10
    Averaging frame 11 into group 11
Integration 26:
    Averaging frame 0 into gro

    Averaging frame 3 into group 3
    Averaging frame 4 into group 4
    Averaging frame 5 into group 5
    Averaging frame 6 into group 6
    Averaging frame 7 into group 7
    Averaging frame 8 into group 8
    Averaging frame 9 into group 9
    Averaging frame 10 into group 10
    Averaging frame 11 into group 11
Integration 50:
    Averaging frame 0 into group 0
    Averaging frame 1 into group 1
    Averaging frame 2 into group 2
    Averaging frame 3 into group 3
    Averaging frame 4 into group 4
    Averaging frame 5 into group 5
    Averaging frame 6 into group 6
    Averaging frame 7 into group 7
    Averaging frame 8 into group 8
    Averaging frame 9 into group 9
    Averaging frame 10 into group 10
    Averaging frame 11 into group 11
Integration 51:
    Averaging frame 0 into group 0
    Averaging frame 1 into group 1
    Averaging frame 2 into group 2
    Averaging frame 3 into group 3
    Averaging frame 4 into group 4
    Averaging frame 5 into group 5
    Averaging f

WCS info for /Users/thatte/niriss/dat_pyinthesky/jdat_notebooks/niriss_ami_binary/mirage_sim_data/jw01093002001_01101_00001_nis_uncal.fits complete.
2020-09-16 10:27:57,172 - stpipe - INFO - WCS info for /Users/thatte/niriss/dat_pyinthesky/jdat_notebooks/niriss_ami_binary/mirage_sim_data/jw01093002001_01101_00001_nis_uncal.fits complete.


Final raw exposure saved to: 
/Users/thatte/niriss/dat_pyinthesky/jdat_notebooks/niriss_ami_binary/mirage_sim_data/jw01093002001_01101_00001_nis_uncal.fits


Segment 1 out of 1 complete.
Observation generation complete.


In [8]:
datafiles = sorted(glob.glob('mirage_sim_data/jw*uncal.fits'))
m = glob.glob("*")
print(datafiles)

['mirage_sim_data/jw01093001001_01101_00001_nis_uncal.fits', 'mirage_sim_data/jw01093002001_01101_00001_nis_uncal.fits']


### Look at the data

In [9]:
for i, df in enumerate(datafiles):
    file = fits.open(df)
    file.info()
    im = file[1].data


Filename: mirage_sim_data/jw01093001001_01101_00001_nis_uncal.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     125   ()      
  1  SCI           1 ImageHDU        50   (80, 80, 5, 65)   int16 (rescales to uint16)   
  2  ZEROFRAME     1 ImageHDU        12   (80, 80, 65)   int16 (rescales to uint16)   
  3  GROUP         1 BinTableHDU     36   325R x 13C   [I, I, I, J, I, 26A, I, I, I, I, 36A, D, D]   
  4  INT_TIMES     1 BinTableHDU     24   65R x 7C   [J, D, D, D, D, D, D]   
  5  ASDF          1 BinTableHDU     11   1R x 1C   [4455B]   
Filename: mirage_sim_data/jw01093002001_01101_00001_nis_uncal.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     125   ()      
  1  SCI           1 ImageHDU        50   (80, 80, 12, 60)   int16 (rescales to uint16)   
  2  ZEROFRAME     1 ImageHDU        12   (80, 80, 60)   int16 (rescales to uint16)   
  3  GROUP         1 BinTableHDU     36   

In [11]:
with fits.open(datafiles[0]) as f:
    f.info()
    #f.close()
    print(" %s"%(os.path.basename(datafiles[0])))
    im = f[1].data
    print(im[0].shape)
    #Examine the last group of an integration
    plt.imshow(im[0][4], origin='lower')

Filename: mirage_sim_data/jw01093001001_01101_00001_nis_uncal.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     125   ()      
  1  SCI           1 ImageHDU        50   (80, 80, 5, 65)   int16 (rescales to uint16)   
  2  ZEROFRAME     1 ImageHDU        12   (80, 80, 65)   int16 (rescales to uint16)   
  3  GROUP         1 BinTableHDU     36   325R x 13C   [I, I, I, J, I, 26A, I, I, I, I, 36A, D, D]   
  4  INT_TIMES     1 BinTableHDU     24   65R x 7C   [J, D, D, D, D, D, D]   
  5  ASDF          1 BinTableHDU     11   1R x 1C   [4455B]   
 jw01093001001_01101_00001_nis_uncal.fits
(5, 80, 80)


## File information

Explain pertinent details about the file you've just downloaded. For example, if working with Kepler lightcurves, explain what's in the different file extensions:

```
- No. 0 (Primary): This HDU contains meta-data related to the entire file.
- No. 1 (Light curve): This HDU contains a binary table that holds data like flux measurements and times. We will extract information from here when we define the parameters for the light curve plot.
- No. 2 (Aperture): This HDU contains the image extension with data collected from the aperture. We will also use this to display a bitmask plot that visually represents the optimal aperture used to create the SAP_FLUX column in HDU1.
```

## Sections (xN)

The meat of your notebook should be sub-divided into sections with useful names to whatever extent makes sense base on the content. Break sections up with standard Markdown syntax headings:

```
# Section 1

Intro to section 1

### Sub-section 1a

More detailed info about Section 1

## Section 2

A complete thought that's as important as section 1 but doesn't need sub-sections.
```

## Aditional Resources
While this isn't always necessary, sometimes you want to provide some more resources for the reader who wants to learn something beyond what's in the notebook. Sometimes these don't exist, but if they do, it's good to put them at the end to give the reader somewhere else to go. Usually a list of links using markdown bullet-plus-link format is appropriate:

- [MAST API](https://mast.stsci.edu/api/v0/index.html)
- [Kepler Archive Page (MAST)](https://archive.stsci.edu/kepler/)
- [Kepler Archive Manual](https://archive.stsci.edu/kepler/manuals/archive_manual.pdf)
- [Exo.MAST website](https://exo.mast.stsci.edu/exo/ExoMast/html/exomast.html)

## About this notebook

**Author:** Deepashri Thatte, Staff Scientist II, Kevin Volk, Associate Scientist, CSA
**Updated On:** 2020-09-15

***

[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 